### import 및 파일 읽기

In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl import Workbook
import warnings
warnings.simplefilter("ignore")
import functions

# 출력 옵션
pd.options.display.float_format = '{:,.0f}'.format

account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}
deposit_dtype = {'채무자키':str, '입금키':str, '계좌키':str, '계좌번호':str, '입금고정키':str, '타채무자키':str}
reduction_dtype = {'채무자키':str, '계좌키':str, '감면키':str}
installment_dtype = {'채무자키':str, '계좌키':str, '분납키':str}

#### 파일읽기 : 첫 기준데이터 만들기

In [3]:
# 파일읽기
##################################
ext = ".xlsx"  
company = "솔림"      # 솔림 or 대성
basedate = "250408" 
# previous_basedate = "250331" # 전달 기준데이터 필요
##################################

기준일 = '20' + basedate[:2] +'-'+basedate[2:4]+'-'+basedate[4:6]
기준일_dt = pd.to_datetime(기준일)
wd = join(r"D:\3.자산\전산 dataset", company)

# 기본 경로
base_dir = join(wd, basedate)
if ext == ".pkl" : 
        base_dir = join(base_dir, "pkl")
        
# 파일검색어
파일검색어 = ["계좌조회새창_", "보증인새창_", "감면조회새창", "약정분납새창"]#, "입금조회새창"]
# 파일명리스트
files = []
for i in range(len(파일검색어)) : 
        if re.search("보증인새창", 파일검색어[i]) : # 새보증인상태 파일 말고 원 파일 읽자
                files.append(os.path.splitext(functions.키워드로파일명찾기(base_dir, 파일검색어[i],'새보증인상태',전체경로=False))[0])  
        elif re.search("약정분납새창", 파일검색어[i]) : # 새보증인상태 파일 말고 원 파일 읽자
                files.append(os.path.splitext(functions.키워드로파일명찾기(base_dir, 파일검색어[i],'기준데이터',전체경로=False))[0])         
        else : 
                files.append(os.path.splitext(functions.키워드로파일명찾기(base_dir, 파일검색어[i],전체경로=False))[0])

if ext == ".pkl" : # fillna()는 엑셀읽을 때 미리 해둬야함, 조정, 개회 진행중 시트만 읽어서 저장한 경우
    account = pd.read_pickle(join(wd, basedate, "pkl", files[0]+ext))
    grt = pd.read_pickle(join(wd, basedate, "pkl", files[1]+ext))
    reduction_ori = pd.read_pickle(join(wd, basedate, "pkl", files[2]+ext))
    installment_ori = pd.read_pickle(join(wd, basedate, "pkl", files[3]+ext))
#     deposit = pd.read_pickle(join(wd, basedate, "pkl", files[4]+ext))
else : 
    account = pd.read_excel(join(wd, basedate, files[0]+ext), dtype=account_dtype).fillna("")
    grt = pd.read_excel(join(wd, basedate, files[1]+ext), dtype=grt_dtype).fillna("")
    reduction_ori = pd.read_excel(join(wd, basedate, files[2]+ext), dtype=reduction_dtype).fillna("")
    installment_ori = pd.read_excel(join(wd, basedate, files[3]+ext), dtype=installment_dtype).fillna("")
#     deposit = pd.read_excel(join(wd, basedate, files[4]+ext), dtype=deposit_dtype).fillna("")

# 보증인새창에 이름키_ 추가 (이름키 - 엄격(pk), 제거시 사용 / 구분키-덜엄격, 남기는 작업시 사용)
grt['이름키_'] = grt['채무자키'] + grt['보증인성명인']

# 전달 기준데이터 (최종입금일 확인위해) : 
# pre_reduction = pd.read_pickle(functions.키워드로파일명찾기(join(wd, previous_basedate,"pkl"), "약정분납새창.*기준데이터")[0])

In [ ]:
# 일부 파일만 다시 읽기
reduction_ori = pd.read_excel(join(wd, basedate, files[2]+ext), dtype=reduction_dtype).fillna("")
installment_ori = pd.read_excel(join(wd, basedate, files[3]+ext), dtype=installment_dtype).fillna("")

##### 미종결계좌 전기간 입금 읽기

In [4]:
계좌종결담당자정규식 = r"(?<!청산)종결|매각(?!예정)|(?<!피)환매(?!예정)|DS|완납|완제|종료|매각\(예정\)|(?<!피)환매\(예정\)|상각" 
계좌미종결건 = account.query('종결일=="" and ~담당자.str.contains(@계좌종결담당자정규식)')[["채무자키","계좌키","종결일","매입일"]]

계좌미종결건 = 계좌미종결건.drop_duplicates(subset='채무자키')
계좌미종결건 = 계좌미종결건.reset_index(drop=True)
계좌미종결건["순번"] = 계좌미종결건.index
계좌미종결건["만단위구분"] = 계좌미종결건["순번"]  // 10000

계좌미종결건[["채무자키","만단위구분"]].to_excel("입금조회용계좌키.xlsx", index=False)
계좌미종결건['매입일'] = pd.to_datetime(계좌미종결건["매입일"]) 
print(계좌미종결건.매입일.min())

2017-07-01 00:00:00


In [5]:
# 입금조회새창들 병합
포함키워드 = company+"_\d\.입금조회새창"
fnlist = functions.키워드로파일명찾기(base_dir, 포함키워드, 여러파일허용=True)
if isinstance(fnlist, str) : # 파일 하나인 경우
    입금 = pd.read_excel(fnlist, dtype=deposit_dtype)
else : # 파일 여러개(리스트)인 경우
    입금 = pd.DataFrame(None)
    for fn in fnlist :
        입금 = pd.concat([입금, pd.read_excel(fn, dtype=deposit_dtype)])

#### 파일읽기 : 이전 기준데이터 있는 경우

In [ ]:
# 파일읽기
##################################
ext = ".xlsx"  
company = "솔림"      # 솔림 or 대성
basedate = "250408" 
previous_basedate = "250331" # 전달 기준데이터 필요
##################################

기준일 = '20' + basedate[:2] +'-'+basedate[2:4]+'-'+basedate[4:6]
wd = join(r"D:\3.자산\전산 dataset", company)

# 기본 경로
base_dir = join(wd, basedate)
if ext == ".pkl" : 
        base_dir = join(base_dir, "pkl")
        
# 파일검색어
파일검색어 = ["계좌조회새창_", "보증인새창_", "감면조회새창", "약정분납새창"]#, "입금조회새창"]
# 파일명리스트
files = []
for i in range(len(파일검색어)) : 
        if re.search("보증인새창", 파일검색어[i]) : # 새보증인상태 파일 말고 원 파일 읽자
                files.append(os.path.splitext(functions.키워드로파일명찾기(base_dir, 파일검색어[i],'새보증인상태',전체경로=False))[0])     
        elif re.search("약정분납새창", 파일검색어[i]) : # 새보증인상태 파일 말고 원 파일 읽자
                files.append(os.path.splitext(functions.키워드로파일명찾기(base_dir, 파일검색어[i],'기준데이터',전체경로=False))[0])       
        else : 
                files.append(os.path.splitext(functions.키워드로파일명찾기(base_dir, 파일검색어[i],전체경로=False))[0])

if ext == ".pkl" : # fillna()는 엑셀읽을 때 미리 해둬야함, 조정, 개회 진행중 시트만 읽어서 저장한 경우
    account = pd.read_pickle(join(wd, basedate, "pkl", files[0]+ext))
    grt = pd.read_pickle(join(wd, basedate, "pkl", files[1]+ext))
    reduction = pd.read_pickle(join(wd, basedate, "pkl", files[2]+ext))
    installment_ori = pd.read_pickle(join(wd, basedate, "pkl", files[3]+ext))
    deposit_ori = pd.read_pickle(join(wd, basedate, "pkl", files[4]+ext))
else : 
    account = pd.read_excel(join(wd, basedate, files[0]+ext), dtype=account_dtype).fillna("")
    grt = pd.read_excel(join(wd, basedate, files[1]+ext), dtype=grt_dtype).fillna("")
    reduction = pd.read_excel(join(wd, basedate, files[2]+ext), dtype=reduction_dtype).fillna("")
    installment_ori = pd.read_excel(join(wd, basedate, files[3]+ext), dtype=installment_dtype).fillna("")
    deposit_ori = pd.read_excel(join(wd, basedate, files[4]+ext), dtype=deposit_dtype).fillna("")

# 보증인새창에 이름키_ 추가
grt['이름키_'] = grt['채무자키'] + grt['보증인성명인']

# 전달 기준데이터 (최종입금일 확인위해) : 
pre_reduction = pd.read_pickle(functions.키워드로파일명찾기(join(wd, previous_basedate,"pkl"), "약정분납새창.*기준데이터")[0])

#### 입금파일 전처리

In [6]:
# 입금파일 전처리
deposit = 입금[["채무자키", "계좌키","입금키", "입금합계","입금일", "입금구분","입금자구분","입금자", "입금기간"]].copy()

# 불필요 행 삭제 
# cond1 = deposit.입금일 <= 기준일
cond2 = deposit.입금구분.str.contains("약정분납|임의변제") # 기타 포함되어야 함
deposit['입금기간'] = deposit['입금기간'].astype(str)
cond3 = ~(deposit.입금기간.str.contains("매입전|매각후|환매|원상회복전|회수제외|오류입금|환불"))

deposit = deposit[cond2 & cond3]

# 구분키_(채무자키+입금자구분)별로 최근입금일_만 남기기 : 다계좌키인 경우, 특정계좌에만 입금등록할 수도 있으므로 계좌를 기준으로 하면 안 됨
        # 입금은 이름이 지멋대로인 경우가 많아서 그냥 구분키를 pk로 쓰자. 보증인들이 동시에 진행하는 경우도 현재는 없다. 250410
        # 구분키를 기준으로 중복행 삭제했기 때문에 다른 곳에서라도 이름키를 사용할 필요도 없다.
deposit["구분키_"] = deposit["채무자키"] + deposit["입금자구분"]
deposit.sort_values(by='입금일', ascending=False, inplace=True)
deposit = deposit.drop_duplicates(subset=['구분키_']) # 지우는 거지만 입금은 구분키로 한다.

print('전체 약정입금건(구분키중복제거) : ',len(deposit))

전체 약정입금건(구분키중복제거) :  762


### 전처리,DC 및 재시작점

In [7]:
# 종결리스트
계좌종결담당자정규식 = r"(?<!청산)종결|매각(?!예정)|(?<!피)환매(?!예정)|DS|완납|완제|종료|매각\(예정\)|(?<!피)환매\(예정\)|상각" 
면책정규식 =  r"면책|면탈|추심제외|포기|폐업|청산|종결|패소"   # 파산하고 사망 일단 제외해봄   파산(?!(\(기각\)|\s누락|\(누락\)))|사망(?!\(상속\))

    # 계좌 종결건(채무자키)
계좌종결건 = account.query('종결일!="" or 담당자.str.contains(@계좌종결담당자정규식)')[["채무자키","계좌키","종결일"]]

    # 차주면책건
차주면책건 = account.query('채무상태.str.contains(@면책정규식)')["채무자키"]

    # 보증인면책건 : 감면과 약정분납은 보증계좌가 아니라도 등록 가능하므로 등록오류가 있을 수 있어 계좌키 아닌 채무자키를 사용하기로 한다.
보증종결건 = grt.query('종결일!="" or 보증인상태.str.contains(@면책정규식)')[["채무자키","계좌키","보증인키","보증인성명인","이름키_", "보증인상태","종결일"]]
보증종결건.loc[보증종결건.종결일=="", "종결일"] = "종결일없는보증종결"


# 출력cols 정의
감면출력cols = ["채무자키","이름키_","감면자구분","변제자관계","변제자명인","채무자명","변제방법","결재상태","승인일","입금합계"] # 감면자는 cols에 없음
분납출력cols = ["채무자키","이름키_","분납자관계","분납자성명인","채무자명","총분납회차","총분납금","총분납입금","총분납잔금","선납금","분납메모", "분납비고"] # 분납기타, 기타금액 열 없음

#### 감면

In [8]:
# 키_ 생성 및 원본 복사
reduction_ori["이름키_"] = reduction_ori["채무자키"] + reduction_ori["변제자명인"] # 원래 변제자명인이 아니라 감면자명으로 해야 하는데, 엑셀에는 감면자명이 없음.....
reduction_ori["구분키_"] = reduction_ori["채무자키"] + reduction_ori["감면자구분"]
reduction = reduction_ori.copy()

# 감면리스트 작성
    # 승인완료, 요청건만 남기기
reduction = reduction.query('결재상태=="승인완료" or 결재상태=="승인요청"')
    # 계좌 종결건 제외
reduction = reduction.query('~계좌키.isin(@계좌종결건.계좌키.values)')
    # 보증종결제외
보증종결cond = reduction['이름키_'].isin(보증종결건['이름키_'].values) # 감면자구분 보증인을 채무자로 잘못 등록한 건 50건 정도 있음. 차주와 보증인 이름이 같을 수는 없으니 감면자구분은 조건에 넣지 않음
reduction = reduction.loc[~보증종결cond] # 차주 면책되었으나 보증인 대위하는 경우 있어, 차주면책은 제외하지 않고 dc에서 살핌
print('감면건수 :',len(reduction))

감면건수 : 249


In [9]:
#[DC]
print('차주감면 : 변제자명이 차주명과 다름')
display(reduction.query('감면자구분=="채무자" and 채무자명!=변제자명인')[감면출력cols])

print('보증감면 : 변제자명이 보증인중에 없음')
display(reduction[(reduction["감면자구분"] == "보증인") &(~reduction["이름키_"].isin(grt["이름키_"]))][감면출력cols])

# 감면자가 차주인 걸 확실히 하려면 변제자관계가 보증인이 아닌 것까지 봐야.
print('차주면책인데, 차주가 변제중인건')
차주면책cond = (reduction['감면자구분']=="채무자")&(reduction['변제자관계']!="보증인") & (reduction['채무자키'].isin(차주면책건))
display(reduction.loc[차주면책cond][감면출력cols])

# 변제예정액, 변제방법이 없는 건
print('입금합계, 변제방법 없음')
display(reduction[(reduction['입금합계']==0) | (reduction["변제방법"]=="")][감면출력cols])

차주감면 : 변제자명이 차주명과 다름


,채무자키,이름키_,감면자구분,변제자관계,변제자명인,채무자명,변제방법,결재상태,승인일,입금합계


보증감면 : 변제자명이 보증인중에 없음


,채무자키,이름키_,감면자구분,변제자관계,변제자명인,채무자명,변제방법,결재상태,승인일,입금합계
610,20462080,20462080장만석,보증인,차주전남편이보증대위,장만석,김은하,분납상환,승인완료,2025-04-09 오후 3:57:03,6000000
615,20408841,20408841강희만,보증인,"차주,보증대위",강희만,강희만,분납상환,승인완료,2025-04-09 오후 4:04:04,6280000


차주면책인데, 차주가 변제중인건


,채무자키,이름키_,감면자구분,변제자관계,변제자명인,채무자명,변제방법,결재상태,승인일,입금합계


입금합계, 변제방법 없음


,채무자키,이름키_,감면자구분,변제자관계,변제자명인,채무자명,변제방법,결재상태,승인일,입금합계


#### 약정분납

In [10]:
# 키생성 및 원본 복사
installment_ori["이름키_"] = installment_ori["채무자키"] + installment_ori["분납자성명인"] # 원래 변제자명인이 아니라 감면자명으로 해야 하는데, 엑셀에는 감면자명이 없음.....
installment_ori["구분키_"] = installment_ori["채무자키"] + installment_ori["분납자관계"]

# 약정분납 진행건, 원본복사
installment = installment_ori.query('분납상태=="진행"').copy()

# 약정분납 계좌분리
installment['계좌비고_'] = ""
installment['계좌키길이'] = installment.계좌키.apply(len)
단일계좌 = installment.query('계좌키길이 == 9')
다계좌 = installment.query('계좌키길이 > 9')

new_rows = []
for index, row in 다계좌.iterrows():
    account_keys = row['계좌키'].split(',')
    first_account_key = account_keys.pop(0)
    다계좌.loc[index, '계좌키'] = first_account_key # 여러 계좌키를 첫번째 계좌키로 값 수정

    for account_key in account_keys:
        new_row = row.copy() # 일단 모든 칼럼 복사
        new_row['계좌키'] = account_key
                            # OPB도 계좌통합이므로 분리계좌는 0 으로 해줘야
        new_row[['총분납금','총분납입금','분납잔금','총분납잔금', '최초원금','최초법비용','최초미수이자','최초연체이자','최초합계','회수원금','회수법비용',\
            '회수미수이자',	'회수연체이자','회수기타','회수합계','현재원금','현재법비용','현재미수이자','현재연체이자','현재합계']] = 0
        new_row["계좌비고_"] = "통합약정 " + first_account_key
        new_rows.append(new_row)
        

temp = pd.DataFrame(new_rows)

installment_sep = pd.concat([단일계좌, 다계좌, temp], ignore_index=True)

In [11]:
# DC 1 : 분납상태 바꿀 것 출력
종결진행건 = installment_sep.merge(계좌종결건[["채무자키", "종결일"]], on=["채무자키"], how='left')
종결진행건 = 종결진행건.merge(보증종결건[["이름키_", "종결일"]], on=["이름키_"], how='left', suffixes=['','_보증'])
종결진행건[["종결일", "종결일_보증"]] = 종결진행건[["종결일", "종결일_보증"]].fillna("")

분납중단요청건 = 종결진행건[(종결진행건["종결일"]!="") | (종결진행건["종결일_보증"]!="")]
if len(분납중단요청건)>0 :
    print(f'{len(분납중단요청건)}건의 분납상태 중단으로 바꿀 건 vscode폴더에 엑셀출력, 계좌종결일과 사유를 우선반영할 것')
    print('분납조회새창에서 분납상태 일괄 변경 후, 분납파일만 다시 읽고 코드 재실행')
    분납중단요청건[["분납키","채무자키","종결일","종결일_보증","분납자관계","분납자성명인"]].to_excel('분납중단요청건.xlsx', index=False)
else : 
    print('분납상태 중단으로 바꿀 것이 없음')

분납상태 중단으로 바꿀 것이 없음


In [12]:
# DC 2 : 기타

# 승인완료건중 분납없는 건
print('승인완료중 분납없는 건')
display(reduction[~reduction['이름키_'].isin(installment_sep['이름키_']) & (reduction['결재상태']=='승인완료')][감면출력cols])

# 감면입금액과 분납입금액 다른 건
    # 감면과 약정분납 모두 이름키_별로 통합해야 함 (약정분납은 계좌분리전 상태를 사용하면됨)
        # 감면 이름키_별로 입금합계 통합
감면입금합계 = reduction.copy()
감면입금합계['약정키별입금합계'] = 감면입금합계.groupby('이름키_')['입금합계'].transform('sum')
감면입금합계 = 감면입금합계.drop_duplicates(subset=['이름키_'])
    # 병합
감면입금합계 = 감면입금합계.merge(installment[["이름키_", '총분납금']], how='left')

print('감면입금액과 분납입금액 다른 건')
display(감면입금합계.query('약정키별입금합계!=총분납금')[감면출력cols + ['총분납금']])

# 약정분납에 선납금 등록건 (분납메모에 매입전입금액과 같으면 ok)
print('약정분납에 선납금 등록건 (분납메모에 매입전입금액과 같으면 ok)')
display(installment.query('선납금>0')[분납출력cols])

# 분납자명 체크
print('차주분납 : 분납자명인이 차주명과 다름')
display(installment.query('분납자관계=="채무자" and 채무자명!=분납자성명인')[분납출력cols])

print('보증분납 : 보증인중에 해당 이름이 없음')
display(installment[(installment["분납자관계"] == "보증인") &(~installment["이름키_"].isin(grt["이름키_"]))][분납출력cols])

승인완료중 분납없는 건


,채무자키,이름키_,감면자구분,변제자관계,변제자명인,채무자명,변제방법,결재상태,승인일,입금합계
30,20427783,20427783박광식,보증인,보증인,박광식,천민숙,분납상환,승인완료,2025-03-31 오후 1:09:24,2500000
610,20462080,20462080장만석,보증인,차주전남편이보증대위,장만석,김은하,분납상환,승인완료,2025-04-09 오후 3:57:03,6000000
1480,20422759,20422759이영대,채무자,본인,이영대,이영대,분납상환,승인완료,2025-04-09 오후 3:31:18,19100000


감면입금액과 분납입금액 다른 건


,채무자키,이름키_,감면자구분,변제자관계,변제자명인,채무자명,변제방법,결재상태,승인일,입금합계,총분납금
14,20427783,20427783박광식,보증인,보증인,박광식,천민숙,분납상환,승인완료,2025-03-31 오후 1:09:24,2500000,NaN
140,20462080,20462080장만석,보증인,차주전남편이보증대위,장만석,김은하,분납상환,승인완료,2025-04-09 오후 3:57:03,6000000,NaN
197,20422759,20422759이영대,채무자,본인,이영대,이영대,분납상환,승인완료,2025-04-09 오후 3:31:18,19100000,NaN


약정분납에 선납금 등록건 (분납메모에 매입전입금액과 같으면 ok)


,채무자키,이름키_,분납자관계,분납자성명인,채무자명,총분납회차,총분납금,총분납입금,총분납잔금,선납금,분납메모,분납비고
658,20428841,20428841김홍대,채무자,김홍대,김홍대,88,8800000,4900000,3900000,620,"총1500, 매입전620",
666,20429231,20429231강성일,채무자,강성일,강성일,88,8800000,4900000,3900000,320,"총1200, 매입전320",
1253,20552753,20552753김기용,보증인,김기용,박정순,38,7600000,2400000,5200000,7400000,"총1500, 740매입전입금",


차주분납 : 분납자명인이 차주명과 다름


,채무자키,이름키_,분납자관계,분납자성명인,채무자명,총분납회차,총분납금,총분납입금,총분납잔금,선납금,분납메모,분납비고
134,20408465,20408465최상혁(동신물류),채무자,최상혁(동신물류),최상혁,97,38600000,11150000,27450000,0,,
536,20422759,20422759이영대(개명전:이영대(개명전,채무자,이영대(개명전:이영대(개명전,이영대,64,19100000,5250000,13850000,0,,


보증분납 : 보증인중에 해당 이름이 없음


,채무자키,이름키_,분납자관계,분납자성명인,채무자명,총분납회차,총분납금,총분납입금,총분납잔금,선납금,분납메모,분납비고


### 약정분납 기준데이터
- 최종거래일 : 계좌조회창의 최종거래일과 동일(법원환급금 같은 것도 포함). 따라서 최근입금일_로 대체
- 분납연체일 : 입금스케줄상 미납회차에 따름 (조회일 - 미납회차입금일). 따라서 최근입금일_로 대체
- 구분키 : 입금데이터 작업시 구분키 사용. 입금자는 중구난방 많아서 애초에 구분키로 중복제거까지 해버림(250410 현재 문제 없는 거까지 확인함)

##### 진행중 기준데이터(자산정리용)

In [16]:
# 칼럼 정리
installment_sep["반영OPB"] = installment_sep["총분납잔금"]
cols = ['채무자키','계좌키','매각사구분','원채권사','담당자', '분납자관계', '분납자성명인','반영OPB','총분납금', '총분납입금', '총분납잔금', 
        '총분납회차', '잔여회차','최근입금일_', '입금연체일_','분납메모', '분납비고', '계좌비고_','분납시작일', '분납종료일','OPB', '선납금', '분납키', '이름키_', '구분키_'] 
        # '분납상태', '분납중단일', '중단사유',

In [17]:
# 최근입금일_ 불러오기
installment_sep["구분키_"] = installment_sep["채무자키"] + installment_sep["분납자관계"]
    # 병합 후 rst 생성
installment_sep_rst = installment_sep.merge(deposit[["구분키_","입금일"]], how='left')
installment_sep_rst.rename(columns={"입금일":"최근입금일_"}, inplace=True)
    # 날짜타입지정
installment_sep_rst["분납시작일"] = pd.to_datetime(installment_sep_rst["분납시작일"], errors='coerce')
installment_sep_rst["최근입금일_"] = pd.to_datetime(installment_sep_rst["최근입금일_"], errors='coerce') # 최종거래일은 약정분납 입금이 아닌 것도 포함해버림


    # 분납시작일 이전 입금은 제외 (분납종료일 이후에는 입금가능)
nat_cond = installment_sep_rst["최근입금일_"].isna() | (installment_sep_rst["최근입금일_"] < installment_sep_rst["분납시작일"]) # 첫번째 조건 안 쓰면 nat가 아니라 None가 됨.
                # Series.where 사용 (조건을 만족하면 유지, 아니면 NaT로 변경), np.where는 타입을 통일하려 하기때문에 nat와 date가 같이 있으면 date를 int로 바꿔버림
installment_sep_rst["최근입금일_"] = installment_sep_rst["최근입금일_"].where(~nat_cond, pd.NaT) 

# 입금연체일 작성 : 최근입금일_ 이후 기준일까지의 
입금연체일_conds = [
    installment_sep_rst.분납시작일 > 기준일_dt,
    installment_sep_rst.최근입금일_.isna(),
    True
]
입금연체일_values = [
    "상환일미도래",
    (기준일_dt - installment_sep_rst.분납시작일).dt.days,
    (기준일_dt - installment_sep_rst.최근입금일_).dt.days
]
installment_sep_rst["입금연체일_"] = np.select(입금연체일_conds, 입금연체일_values)

In [29]:
# dc 입금관련
# 연체일 너무 긴 것 확인
연체기준일 = 180
연체일있음 = installment_sep_rst[installment_sep_rst["입금연체일_"].apply(lambda x: isinstance(x, int))]
print(f'연체일이 {연체기준일}을 넘어감')
display(연체일있음[연체일있음["입금연체일_"] > 180][cols])
# 담당자 확인
담당자관리팀일반 = r'신용회복|개인회생|파산|예수금|테스트|특이|포기|상각|정현화'
print('분납건 담당자 전체 목록, 추심팀 아닌 목록 확인')
print(installment_sep_rst.담당자.sort_values().unique())
print('담당자 추심팀 아님')
display(installment_sep_rst[installment_sep_rst.담당자.str.contains(담당자관리팀일반)][cols])

연체일이 180을 넘어감


,채무자키,계좌키,매각사구분,원채권사,담당자,분납자관계,분납자성명인,반영OPB,총분납금,총분납입금,...,분납메모,분납비고,계좌비고_,분납시작일,분납종료일,OPB,선납금,분납키,이름키_,구분키_


분납건 담당자 전체 목록, 추심팀 아닌 목록 확인
['김대홍' '김순희' '김효중' '도현관' '박기영' '송경숙' '송춘호' '신경용' '신용회복' '오승열' '이은미' '정종운'
 '파산' '한웅희']
담당자 추심팀 아님


,채무자키,계좌키,매각사구분,원채권사,담당자,분납자관계,분납자성명인,반영OPB,총분납금,총분납입금,...,분납메모,분납비고,계좌비고_,분납시작일,분납종료일,OPB,선납금,분납키,이름키_,구분키_
95,20419670,200935727,한울가람,석진상호신용금고,파산,보증인,김홍식,10000000,10000000,0,...,,,,2025-03-24,2025-03-24,132930556,0,20183738,20419670김홍식,20419670보증인
232,20480116,201001373,롯데카드-04,롯데카드,신용회복,채무자,문정준,1600000,1600000,0,...,,,,2023-04-28,2023-11-28,1857528,0,20143746,20480116문정준,20480116채무자


In [ ]:
# # 담당자관리팀일반 제외 후 저장
# installment_sep_rst = installment_sep_rst[~installment_sep_rst.담당자.str.contains(담당자관리팀일반)]

In [ ]:
# 기준데이터 출력
fn_출력 = files[3] +"_기준데이터_계좌키분리"

        # 정렬
installment_sep_rst = installment_sep_rst.sort_values(by=["분납자관계","분납자성명인","채무자키","총분납금","계좌키"], ascending=[False,True,True,False,True])
        # 타입
def convert_to_int_or_keep(x):
    try:
        # 실수 문자열이라면 → float 변환 후 int
        return int(float(x))
    except:
        # 변환 불가한 일반 문자열은 그대로 유지
        return x
installment_sep_rst["입금연체일_"] = installment_sep_rst["입금연체일_"].apply(convert_to_int_or_keep)


# 저장
functions.save_df_to_excel_two_underline(installment_sep_rst[cols],join(base_dir, fn_출력+".xlsx"), 1,2)
installment_sep_rst[cols].to_pickle(join(base_dir, "pkl", fn_출력+".pkl"))

##### 전체데이터(분납+감면, 상태및담당자 검수용)

In [ ]:
# [상태및담당자 체크용]약정분납 전체
분납전체 = installment_ori.copy()
    # 진행건 최종데이터 외에는 모두 중단으로 변경하기
분납전체["분납상태"] = np.where(분납전체.분납키.isin(installment_sep_rst.분납키), "진행", "중단")
    # 분납시작일 역정렬 후 이름키로 중복제거
분납전체['분납시작일'] = pd.to_datetime(분납전체['분납시작일'])
분납전체 = 분납전체.sort_values(by='분납시작일', ascending=False)
    # 이름키로 중복제거
분납전체 = 분납전체.drop_duplicates(subset='이름키_')

# 감면 전체
감면전체 = reduction_ori.copy()[["채무자키","이름키_","요청일","결재상태"]]
    # 요청일 역정렬 후 이름키로 중복제거
감면전체['요청일'] = 감면전체['요청일'].str[:10]
감면전체['요청일'] = pd.to_datetime(감면전체['요청일'], errors='coerce')
감면전체 = 감면전체.sort_values(by='요청일', ascending=False)
    # 이름키로 중복제거
감면전체 = 감면전체.drop_duplicates(subset='이름키_')

# 분납에 감면 병합
print(f'병합전 분납건 : {len(분납전체)}, 감면건 : {len(감면전체)}, 단순합계 : {len(분납전체)+len(감면전체)}')
약정전체 = 분납전체.merge(감면전체, on=['이름키_', '채무자키'], how='outer')
약정전체.rename(columns={'요청일':'감면요청일', '결재상태':'감면결재상태'})
print(f'병합후 약정건 : {len(약정전체)}')

약정전체.to_pickle(join(base_dir,"pkl","[상태DC]약정_"+basedate+".pkl")) # pkl 로만 저장

병합전 분납건 : 1215, 감면건 : 5201, 단순합계 : 6416
병합후 약정건 : 5583


### [최종DC]

#### 분납미등록건 확인

In [30]:
#################
최초작업 = True  # 첫 기준데이터 만들때 실행하는 코드
#################
if 최초작업 : 
    입금출력cols = ["채무자키","입금키","구분키_","입금자", "입금자구분", "입금일"]
    # 1) 입금일 1년 이상 경과건은 제외
    deposit["입금일"] = pd.to_datetime(deposit["입금일"], errors='coerce')
    deposit_year = deposit[deposit.입금일 >= (기준일_dt - pd.DateOffset(years=1))]
    print('1년내 약정입금건 :',len(deposit_year))

    # 2) 입금이 있는데 분납등록 안 된 경우
    deposit_year['분납등록여부_'] = np.where(deposit_year["구분키_"].isin(installment_sep["구분키_"]), "등록", "미등록")

    분납미등록입금 = deposit_year.query('분납등록여부_=="미등록"')[입금출력cols]
    # 분납미등록입금.to_excel('분납미등록입금_진행건만고려.xlsx', index=False) # 중단건 오류없이 제거되는 거 확인됨. 중단건 제거전 파일은 출력 불요
    # print('분납미등록입근건 :',len(분납미등록입금))


    # 3) 분납중단건(중복제거) : 제거는 엄격하게 성명까지 본다
    installment_end = installment_ori.query('분납상태=="중단"').copy()

    # 분납종결일 순서로 역정렬 후 최근것만 남기기
    installment_end = installment_end.sort_values(by='분납중단일', ascending=False)
    installment_end = installment_end.drop_duplicates(subset=['이름키_']) # 구분키 x, 입금내역과 병합시에는 구분키를 사용하게 되나, 이 경우 최근입금이 더 잘 반영되므로 직접 검수 목록이 늘어나는 것이니 문제 없다.
    print('분납중단건수', len(installment_end))

    # 최근입금내역에서 중단일 이전건은 제거
        # 분납미등록입금에 분납중단일과 분납자 병합
    분납미등록입금 = 분납미등록입금.merge(installment_end[["구분키_","분납중단일","분납자성명인"]], on='구분키_', how='left') # 병합 후 건수 문제 없음 확인완료
        # 타입 날짜로
    분납미등록입금["입금일"] = pd.to_datetime(분납미등록입금["입금일"], errors="coerce")
    분납미등록입금["분납중단일"] = pd.to_datetime(분납미등록입금["분납중단일"], errors="coerce")

    # 중단여부칼럼 작성
    중단전입금조건 = (분납미등록입금["분납중단일"].notna()) & (분납미등록입금["입금일"]<=분납미등록입금["분납중단일"])
    이름일치조건 = 분납미등록입금["입금자"] == 분납미등록입금["분납자성명인"]

    중단여부conds = [
        중단전입금조건 & 이름일치조건,
        중단전입금조건,
    ]

    중단여부values = [
        "중단",
        "확인필요",
    ]
    중단여부_default_value = "진행" # 대부분 (감면반려, 약정분납 미등록건)
    분납미등록입금["중단여부_"] = np.select(중단여부conds, 중단여부values, default=중단여부_default_value)

    분납미등록입금_중단제외 = 분납미등록입금.query('중단여부_!="중단"')

# 엑셀출력
print('분납미등록입금_중단제외 :', len(분납미등록입금_중단제외))
분납미등록입금_중단제외.to_excel('분납미등록입금_중단제외.xlsx', index=False)

1년내 약정입금건 : 353
분납중단건수 931


#### 채무상태와 담당자

In [ ]:
# 정상 채무자키 관리 목록 만들기
차주 = account[["채무자키","계좌키","채무자명","채무자주민번호","성별","채무상태","담당자"]]
    # 종결제거
차주 = 차주[~차주.계좌키.isin(계좌종결건.계좌키)]
#     # 차주면책 제거
# 차주 = 차주[~차주.채무자키.isin(차주면책건)] # 차주면책건은 채무자키만 있음
    # 채무자키 중복제거
차주 = 차주.drop_duplicates(subset='채무자키')
    # 정렬
차주 = 차주.sort_values(by=['채무자주민번호', '채무상태','담당자'])
    # 이름키 작성
차주['이름키_'] = 차주['채무자키'] + 차주['채무자명']

# 차주 목록에 분납상태 넣기
    # 분납진행, 실효,면책 이름키 중복제거
분납진행건 = installment_sep_rst.drop_duplicates(subset='이름키_')

분납면책건 = installment_end[installment_end.중단사유.isin(['납부완료','면책','만기완제','중도완제'])]
분납면책건 = 분납면책건.drop_duplicates(subset='이름키_')

분납실효건 = installment_end[~installment_end.중단사유.isin(['납부완료','면책','만기완제','중도완제'])]
분납실효건 = 분납실효건.drop_duplicates(subset='이름키_')
    
# 차주 목록에 분납결과 작성하기
분납결과conds = [차주.이름키_.isin(분납면책건.이름키_), 차주.이름키_.isin(분납진행건.이름키_), 차주.이름키_.isin(분납실효건.이름키_)] # 순서 주의
분납결과values = ["약정(면책)", '약정(확정)', '약정(실효)']
차주['분납결과'] = np.select(분납결과conds, 분납결과values, default="")

# 엑셀출력 후 확인하기
차주.to_excel(join(base_dir,'[DC]분납결과와 상태및담당자.xlsx'), index=False)

In [43]:
reduction

,채무자키,매각사구분,감면키,채무자명,계좌번호,결재상태,감면상태,감면자구분,변제자명인,변제자관계,...,입금원금,입금법비용,입금미수이자,입금연체이자,입금합계,원금감면율,이자감면율,계좌키,변제방법,이름키_
0,20466377,우담-01,20011048,신승미,459-61-76-0016650,승인완료,,채무자,신승미,본인,...,2100000,0,0,0,2100000,22,100,200986427,분납상환,20466377신승미
1,20466376,우담-01,20011046,신승미,459-01-76-0505845,승인완료,,채무자,신승미,본인,...,3200000,0,0,0,3200000,28,100,200986426,분납상환,20466376신승미
2,20415152,DNP-02,20011044,장두형,TYM-13-1-03928-01,승인완료,,채무자,장두형,본인,...,1100000,0,0,0,1100000,18,100,200925628,분납상환,20415152장두형
6,20464430,우담-01,20011039,김순영,225737,승인완료,,채무자,김순영,본인,...,2126448,73552,0,0,2200000,13,100,200984480,일시상환,20464430김순영
7,20418119,DNP-02,20011035,김현추,TYM-2013-0553-001,승인완료,,채무자,김현추,본인,...,4581850,64300,4853850,0,9500000,0,78,200931777,분납상환,20418119김현추
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3100,20420418,한울가람,20006892,박순민,RFC-B1-03811,승인완료,,채무자,박순민,본인,...,0,0,0,550000,550000,100,88,200937129,일시상환,20420418박순민
3492,20428578,케이에스,20006419,정상빈,005912001139057000,승인완료,,채무자,정상빈,본인,...,8563782,436218,0,0,9000000,42,100,200948174,분납상환,20428578정상빈
3638,20426748,대산,20006284,안병규,26914288818631,승인완료,,채무자,안병규,본인,...,4923360,76640,0,0,5000000,21,100,200945587,분납상환,20426748안병규
4025,20411278,DNP-02,20005872,윤한일,TYM-13-1-03923-01,승인완료,,보증인,신경애,보증인,...,5321464,0,0,0,5321464,6,100,200925626,분납상환,20411278신경애
